In [ ]:
# Imports the libraries
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import sklearn as skl
from datetime import datetime
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import pandas as pd
import keras
%matplotlib inline
from tensorflow.keras.preprocessing.image import img_to_array, load_img
# pip3 install adabelief-tf==0.2.0 #Installs the AdaBelief optimizer from https://github.com/juntang-zhuang/Adabelief-Optimizer#2-tensorflow-implementation-eps-of-adabelief-in-tensorflow-is-larger-than-in-pytorch-same-for-adam (nowadays this optimizer is available on TensorFlow Addons)
from adabelief_tf import AdaBeliefOptimizer
from matplotlib.pyplot import *

In [ ]:
# Checks if the GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# Path for the training, validation, test, checkpoint(used by the callback to save the model during the training), and logs(to be used by TensorBoard)
train_path = '/dataset/train'
validation_path = '/dataset/val'
test_path = '/dataset/test'
checkpoint_path = '/checkpoint_model'
logdir = '/tensorboard_logs/'  + datetime.now().strftime("%d%m%Y-%H%M%S")

size=128 # image size
batch=128 # batch size

train_ds = tf.keras.utils.image_dataset_from_directory(
  train_path,
  batch_size=batch,
  image_size=(size, size),
  seed=123
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  validation_path,
  batch_size=batch,  
  image_size=(size, size),
  seed=123
)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  batch_size=batch,
  image_size=(size, size),
  seed=123
)

In [ ]:
# Shows 9 image examples and their labels
class_names = test_ds.class_names
print(class_names)

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
# Buffered prefetching with automatic parameters for the training, validation, and test
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
callback=[
    EarlyStopping(monitor='val_loss', patience=10), # Interrupts the training if the validation loss is not lower after 10 epochs
    TensorBoard(log_dir=logdir), # Saves training logs in a format used by TensorBoard
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7), # Changes the learning rate in a 0.2 factor every 3 epochs which the validation loss did not decreased until 0.0000001
    ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0) # Saves the best model (based on the lowest validation_loss)
]


# triangular_cyclical_lr=tfa.optimizers.TriangularCyclicalLearningRate(initial_learning_rate=1e-1, maximal_learning_rate=1e-3, step_size=224, scale_mode='cycle')


model = tf.keras.Sequential([
  layers.Rescaling(1./255),
  layers.Conv2D(64, 3, padding='same', activation=tfa.activations.mish),
  layers.BatchNormalization(),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation=tfa.activations.mish),
  layers.BatchNormalization(),
  layers.MaxPooling2D(),
  layers.Conv2D(16, 3, padding='same', activation=tfa.activations.mish),
  layers.BatchNormalization(),
  layers.MaxPooling2D(),
  layers.Conv2D(8, 3, padding='same', activation=tfa.activations.mish),
  layers.BatchNormalization(),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dropout(0.3),
  layers.Dense(256, activation=tfa.activations.mish),
  layers.BatchNormalization(),
  layers.Dropout(0.7),
  layers.Dense(3)
])


model.compile(
  optimizer = AdaBeliefOptimizer(learning_rate=1e-3, epsilon=1e-5, rectify=True, print_change_log = False),
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy', tfa.metrics.CohenKappa(num_classes=3, sparse_labels=True)]
)



model.fit(
  train_ds,
  validation_data=val_ds,
  callbacks=[callback],
  epochs=100
)

In [ ]:
model.evaluate(test_ds) # Evaluates the model on the test dataset

In [ ]:
model = tf.keras.models.load_model("/tf/tcc/model.h5", custom_objects={"AdaBeliefOptimizer": AdaBeliefOptimizer})

In [ ]:
model.summary() # Shows the model parameters

In [ ]:
# Generates a Confusion Matrix
y_pred = model.predict(test_ds)
predicted_categories = tf.argmax(y_pred, axis=1) #y_pred
true_categories = tf.concat([y for x, y in test_ds], axis=0) #y_true
labels = ['COVID19', 'Normal', 'Pneumonia']

# Generates a Normalized Confusion Matrix
cm = confusion_matrix(true_categories, predicted_categories, normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(cmap=plt.cm.Greens, colorbar=False)
plt.title('Normalized Confusion Matrix')
plt.savefig("ds_normalized_matrix.png", format="png", dpi=300)
plt.show()

In [ ]:
# Shows Precision, Recall, F1-Score, Macro e Weighted Average of the model tested on the "test" dataset
print(classification_report(true_categories, predicted_categories, target_names=labels))

In [ ]:
# Loads images to be used by the Grad-CAM

image1 = "/dataset/test/Normal/18579_test.png" 
image2 = "/dataset/test/Normal/16324_test.png" 
image3 = "/dataset/test/Normal/18079_test.png" 

image4 = "/dataset/test/Pneumonia/4374_test.png" 
image5 = "/dataset/test/Pneumonia/11444_test.png"  
image6 = "/dataset/test/Pneumonia/8985_test.png"  

image7 = "/dataset/test/COVID19/1_test.png" 
image8 = "/dataset/test/COVID19/2158_test.png"
image9 = "/dataset/test/COVID19/2769_test.png"

In [ ]:
#Neural network intermediate activations visualization, adapted from: https://colab.research.google.com/github/fchollet/deep-learning-with-python-notebooks/blob/master/chapter09_part03_interpreting-what-convnets-learn.ipynb#scrollTo=K9vtVj0iqnB9

img_path = image1

def get_img_array(img_path, target_size):
    img = load_img(
    img_path, target_size=target_size)
    array = img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

img_tensor = get_img_array(img_path, target_size=(128, 128))


plt.axis("off")
plt.imshow(img_tensor[0].astype("uint8"))
plt.show()

layer_outputs = []
layer_names = []
for layer in model.layers:
    if isinstance(layer, (layers.Conv2D, layers.GlobalMaxPool2D)):
        layer_outputs.append(layer.output)
        layer_names.append(layer.name)
activation_model = keras.Model(inputs=model.input, outputs=layer_outputs)

activations = activation_model.predict(img_tensor)
first_layer_activation = activations[0]
print(first_layer_activation.shape)

images_per_row = 8
for layer_name, layer_activation in zip(layer_names, activations):
    n_features = layer_activation.shape[-1]
    size = layer_activation.shape[1]
    n_cols = n_features // images_per_row
    display_grid = np.zeros(((size + 1) * n_cols - 1,
                             images_per_row * (size + 1) - 1))
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_index = col * images_per_row + row
            channel_image = layer_activation[0, :, :, channel_index].copy()
            if channel_image.sum() != 0:
                channel_image -= channel_image.mean()
                channel_image /= channel_image.std()
                channel_image *= 64
                channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype("uint8")
            display_grid[
                col * (size + 1): (col + 1) * size + col,
                row * (size + 1) : (row + 1) * size + row] = channel_image
    scale = 1. / size
    plt.figure(dpi=100,figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.axis("off")
    plt.imshow(display_grid, aspect="auto", cmap="viridis")

In [ ]:
# Grad-CAM code, adapted from: https://github.com/keras-team/keras-io/blob/master/examples/vision/grad_cam.py


img_path = image9
size=128

# Loads the image and formats it to be received by the model input
def get_img_array(img_path, target_size):
    img = load_img(
    img_path, target_size=target_size)
    array = img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

img_tensor = get_img_array(img_path, target_size=(128, 128))

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # Creates a new model that maps the last convolution layer activations
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # The gradient of the class with higher probability is calculated
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # Gradient of the output neuron pertaining to the feature map of the last convolutional layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # Vector where each input consists of the average gradient intensity relative to a specific channel of the feature map
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Each channel of the activation map is multiplied by the "degree of importance" of the channel in relation to the class with the highest probability, and at the end, all channels are summed in order to obtain the heatmap
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # The heatmap scale is normalized between 0 and 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def save_and_display_gradcam(img_path, heatmap, cam_path="2769.png", alpha=0.4):
    # Loads the image
    img = tf.keras.preprocessing.image.load_img(img_path)
    img = tf.keras.preprocessing.image.img_to_array(img)

    # Rescales the heatmap in a scale between 0 and 255
    heatmap = np.uint8(255 * heatmap)

    # Uses the "jet" colormap for the heatmap
    jet = cm.get_cmap("jet")

    # Uses RGB values for the colormap/heatmap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Generates an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimposes the heatmap to the input image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    # Saves the image with the superimposed heatmap
    superimposed_img.save(cam_path)

    # Returns the Grad-CAM result superimposed to the original image
    return cam_path

last_conv_layer_name = "conv2d_3"
img_size = (128,128)

# Removes the "softmax" function from the last layer 
model.layers[-1].activation = None

img_array = img_tensor
make_gradcam_heatmap(img_array, model, last_conv_layer_name)
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
cam_path = save_and_display_gradcam(img_path, heatmap)
imshow(plt.imread(cam_path))
plt.tight_layout()
plt.axis("off")
plt.show()